In [47]:
#http://www.football-data.co.uk/englandm.php(source)
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import math
import csv

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

## Dataset

In [48]:
epl1617 = "./EPL16:17.csv"
df1617 = pd.read_csv(epl1617)

epl1516 = "./EPL 15:16.csv"
df1516 = pd.read_csv(epl1516)

## List of all the teams

In [49]:
teamList = ["Arsenal", "Bournemouth", "Burnley", "Chelsea", "Crystal Palace", "Everton", "Huddersfield", "Leicester", "Liverpool", "Man City", "Man United", "Brighton", "Southampton", "Stoke", "Newcastle United", "Swansea", "Tottenham", "Watford", "West Brom", "West Ham"]
team_names = pd.DataFrame(teamList, columns=['Team_name'])
(team_names[team_names["Team_name"]  == "Liverpool"])

,Team_name
8,Liverpool


## Creating winner and loser column

In [50]:
def win(row):
    if (row["FTR"] == "A"):
        return row["AwayTeam"]
    elif (row["FTR"] == "H"):
        return row["HomeTeam"]
    elif (row["FTR"] == "D"):
        return "None"
    

def lose(row):
    if (row["FTR"] == "A"):
        return row["HomeTeam"]
    elif (row["FTR"]  == "H"):
        return row["AwayTeam"]
    elif (row["FTR"]  == "D"):
        return "None"

df1617["Winner"] = df1617.apply(win, axis=1) 
df1617["Loser"] = df1617.apply(lose, axis=1)

df1516["Winner"] = df1516.apply(win, axis=1) 
df1516["Loser"] = df1516.apply(lose, axis=1)

In [51]:
df1617.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HF,AF,HC,AC,HY,AY,HR,AR,Winner,Loser
0,E0,13/8/16,Burnley,Swansea,0,1,A,0,0,D,...,10,14,7,4,3,2,0,0,Swansea,Burnley
1,E0,13/8/16,Crystal Palace,West Brom,0,1,A,0,0,D,...,12,15,3,6,2,2,0,0,West Brom,Crystal Palace
2,E0,13/8/16,Everton,Tottenham,1,1,D,1,0,H,...,10,14,5,6,0,0,0,0,None,None
3,E0,13/8/16,Hull,Leicester,2,1,H,1,0,H,...,8,17,5,3,2,2,0,0,Hull,Leicester
4,E0,13/8/16,Man City,Sunderland,2,1,H,1,0,H,...,11,14,9,6,1,2,0,0,Man City,Sunderland


In [52]:
df1516.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HF,AF,HC,AC,HY,AY,HR,AR,Winner,Loser
0,E0,8/8/15,Bournemouth,Aston Villa,0,1,A,0,0,D,...,13,13,6,3,3,4,0,0,Aston Villa,Bournemouth
1,E0,8/8/15,Chelsea,Swansea,2,2,D,2,1,H,...,15,16,4,8,1,3,1,0,None,None
2,E0,8/8/15,Everton,Watford,2,2,D,0,1,A,...,7,13,8,2,1,2,0,0,None,None
3,E0,8/8/15,Leicester,Sunderland,4,2,H,3,0,H,...,13,17,6,3,2,4,0,0,Leicester,Sunderland
4,E0,8/8/15,Man United,Tottenham,1,0,H,1,0,H,...,12,12,1,2,2,3,0,0,Man United,Tottenham


## 16/17 dataset

In [53]:
#goals scored in wins and losses
def TeamData(teamName):
    annual_data = df1617
    #num goals scored in wins and losses 
    gamesHome = annual_data[annual_data["HomeTeam"] == teamName]
    totalGoalsScored = gamesHome["FTHG"].sum()
    gamesAway = annual_data[annual_data["AwayTeam"] == teamName]
    totalGames = gamesHome.append(gamesAway)
    numGames = len(totalGames.index)
    #total goals scored
    totalGoalsScored += gamesAway["FTAG"].sum()
    #total goals allowed
    totalGoalsAllowed = gamesHome["FTAG"].sum()
    totalGoalsAllowed += gamesAway["FTHG"].sum()
    #discipline: total red cards, total yellow cards
    totalYellowCards = gamesHome["HY"].sum()
    totalYellowCards += gamesAway["AY"].sum()
    totalRedCards = gamesHome["HR"].sum()
    totalRedCards += gamesHome["AY"].sum() 
    #total fouls 
    totalFouls = gamesHome["HF"].sum()
    totalFouls += gamesAway["AF"].sum()
    #total corners
    totalCorners = gamesHome["HC"].sum()
    totalCorners += gamesAway["AC"].sum()
    #shots per game (spg) = total shots / total games
    totalShots = gamesHome["HS"].sum()
    #avgshots per game
    totalShots += gamesAway["AS"].sum()
    if numGames != 0:
        spg = totalShots / numGames
    # avg shots allowed per game
    totalShotsAgainst = gamesHome["AS"].sum()
    totalShotsAgainst += gamesAway["HS"].sum()
    if numGames != 0:
        sag = totalShotsAgainst / numGames
    #Games Won percentage = Games Won / (Games won + Games Lost)
    gamesWon = annual_data[annual_data["Winner"] == teamName]
    gamesLost = annual_data[annual_data["Loser"] == teamName]
    numGamesWon = len(gamesWon.index)
    numGamesLost = len(gamesLost.index)
    if numGames != 0:
        gamesWonPercentage = numGamesWon / numGames
     #Defense stats
        #Goalie Saves = Shots on Goal - Goal Scored
    #totalShotsonGoal = tsog
    tsog  = gamesHome["HST"].sum()
    tsog += gamesAway["AST"].sum()
    goalieSaves = tsog - totalGoalsAllowed
       #Saves Percentage = Goalie Saves / Shots on Goal
    if tsog != 0:
        savesPercentage = goalieSaves / tsog
        #Saves Ratio = shots on Goal / Goalie Saves
    if goalieSaves != 0:
        savesRatio = tsog / goalieSaves
    
    #offense stats
        #Scoring Percentage = (Scoring Attempts - Goals Scored) / Scorring Attempts
    if totalShots != 0 :
        scoringPercentage = (totalShots - totalGoalsScored) / totalShots
        #Scoring Ratio = Shots on Goal / Goalie Saves
    if tsog != 0:
        scoringRatio = tsog / totalGoalsScored
        
    if numGames == 0: #if team not in dataset
        gamesWon = 0 
        gamesLost = 0
        totalGoalsScored = 0
        totalGoalsAllowed = 0
        totalYellowCards = 0
        totalRedCards = 0
        totalFouls = 0
        totalCorners = 0
        spg = 0
        sag = 0
        gamesWonPercentage = 0
        goalieSaves = 0 
        savesPercentage = 0
        savesRatio= 0
        scoringPercentage = 0 
        scoringRatio = 0 

    return[totalGoalsScored, totalGoalsAllowed, totalYellowCards, totalRedCards, scoringPercentage, totalFouls, totalCorners, spg, sag, gamesWonPercentage,goalieSaves, savesPercentage, savesRatio, scoringRatio]

## 16/ 17 Model training

In [54]:
totalNumGames = 0 
totalNumGames += len(df1617.index)
numFeatures = len(TeamData("Arsenal"))# to find dimensionality
xTrain1 = np.zeros((totalNumGames, numFeatures))
yTrain1 = np.zeros((totalNumGames))                     

In [55]:
from collections import defaultdict
annualDictionary = defaultdict(list)
for team in teamList:
    team_vector = TeamData(team)
    annualDictionary[team] = team_vector

In [56]:
team_vectors = annualDictionary
annual = df1617
numGamesInYear = len(annual.index)
xTrainAnnual = np.zeros((numGamesInYear, numFeatures))
yTrainAnnual = np.zeros((numGamesInYear))
counter = 0  

In [57]:
for index, row in annual.iterrows():
    h_team = row["HomeTeam"]
    h_vector = team_vectors[h_team]
    a_team = row["AwayTeam"]
    a_vector = team_vectors[a_team]
    diff = [a-b for a, b in zip(h_vector, a_vector)]
    if (counter % 2 == 0):
        if len(diff) != 0:
            xTrainAnnual[counter] = diff
        yTrainAnnual[counter] = 1
    else: 
        if len(diff)!=0:
            xTrainAnnual[counter] = [-p for p in diff]
        yTrainAnnual[counter] = 0
    counter += 1

In [58]:
indexCounter = 0 
xTrain1[indexCounter:numGamesInYear+indexCounter] = xTrainAnnual
yTrain1[indexCounter:numGamesInYear+indexCounter] = yTrainAnnual
indexCounter += numGamesInYear

In [59]:
xTrain1.shape
yTrain1.shape

(380,)

## 15/16 dataset

In [60]:
def TeamData2(teamName):
    annual_data = df1516
    #num goals scored in wins and losses 
    gamesHome = annual_data[annual_data["HomeTeam"] == teamName]
    totalGoalsScored = gamesHome["FTHG"].sum()
    gamesAway = annual_data[annual_data["AwayTeam"] == teamName]
    totalGames = gamesHome.append(gamesAway)
    numGames = len(totalGames.index)
    #total goals scored
    totalGoalsScored += gamesAway["FTAG"].sum()
    #total goals allowed
    totalGoalsAllowed = gamesHome["FTAG"].sum()
    totalGoalsAllowed += gamesAway["FTHG"].sum()
    #discipline: total red cards, total yellow cards
    totalYellowCards = gamesHome["HY"].sum()
    totalYellowCards -= gamesAway["AY"].sum()
    totalRedCards = gamesHome["HR"].sum()
    totalRedCards -= gamesHome["AY"].sum() 
    #total fouls 
    totalFouls = gamesHome["HF"].sum()
    totalFouls += gamesAway["AF"].sum()
    #total corners
    totalCorners = gamesHome["HC"].sum()
    totalCorners += gamesAway["AC"].sum()
    #shots per game (spg) = total shots / total games
    totalShots = gamesHome["HS"].sum()
    #avgshots per game
    totalShots += gamesAway["AS"].sum()
    if numGames != 0:
        spg = totalShots / numGames
    # avg shots allowed per game
    totalShotsAgainst = gamesHome["AS"].sum()
    totalShotsAgainst += gamesAway["HS"].sum()
    if numGames != 0:
        sag = totalShotsAgainst / numGames
    #Games Won percentage = Games Won / (Games won + Games Lost)
    gamesWon = annual_data[annual_data["Winner"] == teamName]
    gamesLost = annual_data[annual_data["Loser"] == teamName]
    numGamesWon = len(gamesWon.index)
    numGamesLost = len(gamesLost.index)
    if numGames != 0:
        gamesWonPercentage = numGamesWon / numGames
     #Defense stats
        #Goalie Saves = Shots on Goal - Goal Scored
    #totalShotsonGoal = tsog
    tsog  = gamesHome["HST"].sum()
    tsog += gamesAway["AST"].sum()
    goalieSaves = tsog - totalGoalsAllowed
       #Saves Percentage = Goalie Saves / Shots on Goal
    if tsog != 0:
        savesPercentage = goalieSaves / tsog
        #Saves Ratio = shots on Goal / Goalie Saves
    if goalieSaves != 0:
        savesRatio = tsog / goalieSaves
    
    #offense stats
        #Scoring Percentage = (Scoring Attempts - Goals Scored) / Scorring Attempts
    if totalShots != 0 :
        scoringPercentage = (totalShots - totalGoalsScored) / totalShots
        #Scoring Ratio = Shots on Goal / Goalie Saves
    if tsog != 0:
        scoringRatio = tsog / totalGoalsScored
        
    if numGames == 0: #if team not in dataset
        gamesWon = 0 
        gamesLost = 0
        totalGoalsScored
        totalGoalsAllowed = 0
        totalYellowCards = 0
        totalRedCards = 0
        totalFouls = 0
        totalCorners = 0
        spg = 0
        sag = 0
        gamesWonPercentage = 0
        goalieSaves = 0 
        savesPercentage = 0
        savesRatio= 0
        scoringPercentage = 0 
        scoringRatio = 0 

    return[totalGoalsScored, totalGoalsAllowed, totalYellowCards, totalRedCards, totalFouls, totalCorners, spg, sag, gamesWonPercentage, goalieSaves, savesPercentage, savesRatio, scoringPercentage, scoringRatio]

## 15/ 16 Model training

In [61]:
totalNumGames = 0 
totalNumGames += len(df1516.index)
numFeatures = len(TeamData2("Arsenal"))# to find dimensionality
xTrain2 = np.zeros((totalNumGames, numFeatures))
yTrain2 = np.zeros((totalNumGames))

from collections import defaultdict
annualDictionary = defaultdict(list)
for team in teamList:
    team_vector = TeamData2(team)
    annualDictionary[team] = team_vector

team_vectors = annualDictionary
annual = df1516
numGamesInYear = len(annual.index)
xTrainAnnual = np.zeros((numGamesInYear, numFeatures))
yTrainAnnual = np.zeros((numGamesInYear))
counter = 0 

for index, row in annual.iterrows():
    h_team = row["HomeTeam"]
    h_vector = team_vectors[h_team]
    a_team = row["AwayTeam"]
    a_vector = team_vectors[a_team]
    diff = [a-b for a, b in zip(h_vector, a_vector)]
    if (counter % 2 == 0):
        if len(diff) != 0:
            xTrainAnnual[counter] = diff
        yTrainAnnual[counter] = 1
    else: 
        if len(diff)!=0:
            xTrainAnnual[counter] = [-p for p in diff]
        yTrainAnnual[counter] = 0
    counter += 1

indexCounter = 0 
xTrain2[indexCounter:numGamesInYear+indexCounter] = xTrainAnnual
yTrain2[indexCounter:numGamesInYear+indexCounter] = yTrainAnnual
indexCounter += numGamesInYear

In [62]:
# [totalGoalsScored, totalGoalsAllowed, totalYellowCards, totalRedCards, totalFouls, totalCorners, spg, sag, gamesWonPercentage, goalieSaves, savesPercentage, savesRatio, scoringPercentage, scoringRatio]

In [63]:
xTrain1.shape

(380, 14)

In [64]:
#xtrainfin = [xTrain2,xTrain]
xtrainfin = np.concatenate([xTrain2,xTrain1], axis=0)
xtrainfin.shape

(760, 14)

In [65]:
ytrainfin = np.concatenate([yTrain2,yTrain1], axis=0)
ytrainfin.shape

(760,)

## Feature selection

In [66]:
model1 = LogisticRegression() #for recursive feature elimination
model2 = ExtraTreesClassifier()
model3 = RandomForestClassifier()

In [67]:
rfe = RFE(model1, 9)
rfe2 = rfe.fit(xtrainfin, ytrainfin)
print (rfe2.support_)
print (rfe2.ranking_)

[False False False False False  True  True  True  True  True  True  True
  True  True]
[6 5 4 3 2 1 1 1 1 1 1 1 1 1]


In [68]:
model2.fit(xtrainfin, ytrainfin)
print (model2.feature_importances_)
#[totalGoalsScored, totalGoalsAllowed, totalYellowCards, totalRedCards, scoringPercentage, totalFouls, totalCorners, spg, sag, gamesWonPercentage,goalieSaves, savesPercentage, savesRatio, scoringRatio]

[ 0.06924359  0.06555701  0.07074659  0.07446547  0.07731293  0.07861185
  0.06992592  0.07766917  0.06316279  0.06370082  0.06734894  0.0726009
  0.08039348  0.06926055]


In [69]:
model3.fit(xtrainfin, ytrainfin)
print(model3.feature_importances_)

[ 0.06754704  0.07257759  0.0819871   0.0721082   0.06914628  0.07889006
  0.0727412   0.10321531  0.0695184   0.04261572  0.07523856  0.06217789
  0.07268172  0.05955491]


### selected the following features and train and tested data
totalGoalsScored,
totalGoalsAllowed, 
totalYellowCards, 
totalRedCards,  
totalCorners, 
spg,  
sag, 
savesPercentage,
### Accuracy was lower therefore decided to stick with all

In [70]:
xTrain, X_test, yTrain, y_test = train_test_split(xtrainfin, ytrainfin)

## Leave one-out cross validation used 
#### Said to be the best for predicting sport results

In [71]:
xTrain.shape

(570, 14)

In [72]:
X_test.shape

(190, 14)

In [73]:
y_test.shape

(190,)

In [74]:
yTrain.shape

(570,)

In [75]:
xTraindf = pd.DataFrame(xTrain, columns=['totalGoalsScored', 'totalGoalsAllowed', 'totalYellowCards', 'totalRedCards', 'totalFouls', 'totalCorners', 'spg', 'sag', 'gamesWonPercentage', 'goalieSaves', 'savesPercentage', 'savesRatio', 'scoringPercentage', 'scoringRatio'])                                         

## Predicting Result usng Bayesian Linear Regression

In [80]:
xTraindf.columns
bylr = xTraindf.copy()
bylr.shape
len(yTrain)
bylr['yTrain'] = yTrain

In [82]:
import pymc3 as pm
with pm.Model() as reg:
    pm.glm.GLM.from_formula('yTrain ~ totalGoalsScored + totalGoalsAllowed + totalYellowCards + totalRedCards + totalFouls + totalCorners + spg + sag + gamesWonPercentage + goalieSaves + savesPercentage + savesRatio + scoringPercentage + scoringRatio', bylr)
    trace = pm.sample(1000, cores=2, tune = 1000)

ModuleNotFoundError: No module named 'pymc3'

In [83]:
pm.traceplot(trace)

NameError: name 'pm' is not defined

In [84]:
chain = trace[100:]
pm.traceplot(chain);

NameError: name 'trace' is not defined

In [85]:
ppc = pm.sample_ppc(trace, model=reg, samples=1000)

NameError: name 'pm' is not defined

In [ ]:
pm.plot_posterior(trace)

In [ ]:
model_formula = 'Featuressum = '
for variable in trace.varnames:
    model_formula += ' %0.2f * %s +' % (np.mean(trace[variable]), variable)

' '.join(model_formula.split(' ')[:-1])

In [ ]:
var_dict = {}
for variable in trace.varnames:
    var_dict[variable] = trace[variable]
var_weights = pd.DataFrame(var_dict)
var_means = var_weights.mean(axis=0)
var_means

In [ ]:
X_testt = pd.DataFrame(X_test, columns=['totalGoalsScored', 'totalGoalsAllowed', 'totalYellowCards', 'totalRedCards', 'totalFouls', 'totalCorners', 'spg', 'sag', 'gamesWonPercentage', 'goalieSaves', 'savesPercentage', 'savesRatio', 'scoringPercentage', 'scoringRatio'])                                   

In [ ]:
X_testt['Intercept'] = 1

In [ ]:
X_testt.shape

In [ ]:
names = X_testt.columns[1:]
X_testt = X_testt.loc[:, names]
var_means = var_means[names]

In [ ]:
var_means

In [ ]:
ppc = pm.sample_ppc(trace, model=reg, samples=1000)

In [ ]:
len(trace)

In [ ]:
ppc

In [ ]:
result = []
for i in ppc['y']:
    result.append(i.mean())
# ppc['y'][0].mean()
BLavgpredict = sum(result)/len(result)
print BLavgpredict

# Result Using Standard Machine Learning Models

In [ ]:
def evaluate(xTrain, X_test, yTrain, y_test):
    # Names of models
    model_name_list = ['Linear Regression', 'ElasticNet Regression',
                       'Extra Trees', 'SVM',
                       'Gradient Boosted', 'Baseline', "AdaBoostRegressor"]
#     X_train = X_train.drop(columns='Grade')
#     X_test = X_test.drop(columns='Grade')
    
    # Instantiate the models
    model1 = LinearRegression()
    model2 = ElasticNet(alpha=1.0, l1_ratio=0.5)
    model3 = AdaBoostRegressor()
    model4 = ExtraTreesRegressor(n_estimators=50)
    model5 = SVR(kernel='rbf', degree=3, C=1.0, gamma='auto')
    model6 = GradientBoostingRegressor(n_estimators=20)
#     'Random Forest',
# AB = AdaBoostRegressor()
# AB.fit(xTrain, yTrain)
# ABpredict = AB.predict(X_test)
# ABavgpredict = sum(ABpredict)/len(ABpredict)
# print ABavgpredict
    # Dataframe for results
    results = pd.DataFrame(columns=['mae', 'rmse'], index = model_name_list)
    
    # Train and predict with each model
    for i, model in enumerate([model1, model2, model3, model4, model5, model6]):
        model.fit(xTrain, yTrain)
        predictions = model.predict(X_test)
        
        # Metrics
        mae = np.mean(abs(predictions - y_test))
        rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
        
        # Insert results into the dataframe
        model_name = model_name_list[i]
        results.ix[model_name, :] = [mae, rmse]
    
    # Median Value Baseline Metrics
    baseline = np.median(yTrain)
    baseline_mae = np.mean(abs(baseline - y_test))
    baseline_rmse = np.sqrt(np.mean((baseline - y_test) ** 2))
    
    results.ix['Baseline', :] = [baseline_mae, baseline_rmse]
    
    return results

In [ ]:
results = evaluate(xTrain, X_test, yTrain, y_test)

In [ ]:
# Evalute the MCMC trace and compare to ml models
def evaluate_trace(trace, X_train, X_test, y_train, y_test, model_results):
    
    # Dictionary of all sampled values for each parameter
    var_dict = {}
    for variable in trace.varnames:
        var_dict[variable] = trace[variable]
        
    # Results into a dataframe
    var_weights = pd.DataFrame(var_dict)
    
    # Means for all the weights
    var_means = var_weights.mean(axis=0)
    
    # Create an intercept column
    X_test['Intercept'] = 1
    
    # Align names of the test observations and means
    names = X_test.columns[1:]
    X_test = X_test.ix[:, names]
    var_means = var_means[names]
    
    # Calculate estimate for each test observation using the average weights
    results = pd.DataFrame(index = X_test.index, columns = ['estimate'])

    for row in X_test.iterrows():
        results.ix[row[0], 'estimate'] = np.dot(np.array(var_means), np.array(row[1]))
        
    # Metrics 
    actual = np.array(y_test)
    errors = results['estimate'] - actual
    mae = np.mean(abs(errors))
    rmse = np.sqrt(np.mean(errors ** 2))
    
    print('Model  MAE: {:.4f}\nModel RMSE: {:.4f}'.format(mae, rmse))
    
    # Add the results to the comparison dataframe
    model_results.loc['Bayesian LR', :] = [mae, rmse]
    
    plt.figure(figsize=(12, 8))
    
    # Plot median absolute percentage error of all models
    ax = plt.subplot(1, 2, 1)
    model_results.sort_values('mae', ascending = True).plot.bar(y = 'mae', color = 'r', ax = ax)
    plt.title('Model Mean Absolute Error Comparison'); plt.ylabel('MAE'); 
    plt.tight_layout()
    
    # Plot root mean squared error of all models
    ax = plt.subplot(1, 2, 2)
    model_results.sort_values('rmse', ascending = True).plot.bar(y = 'rmse', color = 'b', ax = ax)
    plt.title('Model RMSE Comparison'); plt.ylabel('RMSE')
    
    return model_results

In [ ]:
all_model_results = evaluate_trace(trace, xTrain, X_testt, yTrain, y_test, results)

In [ ]:
results

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
AB = AdaBoostRegressor()
AB.fit(xTrain, yTrain)
ABpredict = AB.predict(X_test)
ABavgpredict = sum(ABpredict)/len(ABpredict)
print ABavgpredict

In [ ]:
from sklearn.tree import DecisionTreeRegressor
DT = DecisionTreeRegressor()
DT.fit(xTrain, yTrain)
DTpredict = DT.predict(X_test)
DTavgpredict = sum(DTpredict)/len(DTpredict)
print DTavgpredict

In [ ]:
rge = Ridge(alpha=0.05, normalize=True)
rge.fit(xTrain, yTrain)
rgepredict = rge.predict(X_test)
rgeavgpredict = sum(rgepredict)/len(rgepredict)
print rgeavgpredict

In [ ]:
ENreg = ElasticNet(alpha=1, l1_ratio=0.5, normalize=False)
ENreg.fit(xTrain, yTrain)
ENregpredict = ENreg.predict(X_test)
ENregavgpredict = sum(ENregpredict)/len(ENregpredict)
print ENregavgpredict

## Game Prediction

In [ ]:
epl1718 = "./17:18 schedule.csv"
df1718 = pd.read_csv(epl1718)
test = df1718
test["Game_ID"].tail()
test = test.drop(test.index[-1])
test["Game_ID"] = test["Game_ID"].astype(int)
test.head()

In [ ]:
# from sklearn.model_selection import LeaveOneOut 
# loo = LeaveOneOut()
# loo.get_n_splits(xtrainfin)
def createGamePred(team1_vector, team2_vector, xTrain, yTrain):
    xTrain, X_test, yTrain, y_test = train_test_split(xtrainfin, ytrainfin)
    EN = ElasticNet(alpha=1.0, l1_ratio=0.5)
    ENpred = EN.fit(xTrain2, yTrain2)
    diff = np.array([a - b for a, b in zip(team1_vector, team2_vector)])
    predictions = EN.predict(diff.reshape(1,-1))
    return predictions

In [ ]:
def createGamePred(team1_vector, team2_vector, xTrain, yTrain):
    xTrain, X_test, yTrain, y_test = train_test_split(xtrainfin, ytrainfin)
    AB = AdaBoostRegressor()
    AB.fit(xTrain, yTrain)
    diff = np.array([a - b for a, b in zip(team1_vector, team2_vector)])
    predictions = AB.predict(diff.reshape(1,-1))
    return predictions

In [ ]:
def formulatePredictions():
    probs = [[0 for x in range(2)] for x in range(len(test.index))]
    for index, row in test.iterrows():
        game_ID = row["Game_ID"]
        team1_Name = row["HomeTeam"]
        team2_Name = row["AwayTeam"]
        team1_vector = TeamData(team1_Name)
        team2_vector = TeamData(team2_Name)
        prediction = createGamePred(team1_vector, team2_vector, xTrain, yTrain)
        probs[index][0] = game_ID
        probs[index][1] = prediction
    probs = pd.np.array(probs)
    return probs

In [ ]:
# prediction = createGamePred(team1_vector, team2_vector, xTrain1, yTrain1)
# prediction

In [ ]:
predictdf=pd.DataFrame(formulatePredictions(), columns=[["Game_ID","Prediction"]])

In [ ]:
predictdf["Game_ID"] = predictdf["Game_ID"].astype(int)

In [ ]:
test.columns

In [ ]:
predictdf.columns

In [ ]:
predictdf.head()

In [ ]:
pf1 = predictdf.drop(['Game_ID'], axis = 1)

In [ ]:
pf1.head()

In [ ]:
test.head()

In [ ]:
predictable = pd.concat([test, pf1], axis=1)
predictable.columns

In [ ]:
ptdf = predictable.drop(predictable.index[-1])
ptdf

In [ ]:
ptdf2 = ptdf[ptdf.HomeTeam.str.contains("Brighton") == False]
ptdf3 = ptdf2[ptdf2.AwayTeam.str.contains("Brighton") == False]
ptdf4 = ptdf3[ptdf.HomeTeam.str.contains("Huddersfield") == False]
ptdf5 = ptdf4[ptdf2.AwayTeam.str.contains("Huddersfield") == False]
ptdf7 = ptdf6[ptdf6.HomeTeam.str.contains("Newcastle") == False]
ptdf8 = ptdf7[ptdf7.AwayTeam.str.contains("Newcastle") == False]

In [ ]:
ptdf8.columns.values[4] = "HT win Predict"
ptdf8

In [ ]:
dfA = {1: ['11/8/17', 'Arsenal', 'Leicester', 1, 0.5014359082073364],
 3: ['12/8/17', 'Chelsea', 'Burnley', 3, 0.5028806289553723],
 5: ['12/8/17', 'Everton', 'Stoke', 5, 0.49654676380750956],
 6: ['12/8/17', 'Southampton', 'Swansea', 6, 0.5012146211844342],
 7: ['12/8/17', 'Watford', 'Liverpool', 7, 0.49370852926186964],
 8: ['12/8/17', 'West Brom', 'Bournemouth', 8, 0.5082627000510842],
 9: ['13/8/17', 'Man United', 'West Ham', 9, 0.49997421596078157],
 10: ['13/8/17', 'Newcastle', 'Tottenham', 10, 0.5379421799646498],
 11: ['19/8/17', 'Bournemouth', 'Watford', 11, 0.5048566100521162],
 12: ['19/8/17', 'Burnley', 'West Brom', 12, 0.49339188064514644],
 14: ['19/8/17', 'Liverpool', 'Crystal Palace', 14, 0.49819945510568764],
 15: ['19/8/17', 'Southampton', 'West Ham', 15, 0.49722755619396736],
 16: ['19/8/17', 'Stoke', 'Arsenal', 16, 0.4991369759581655],
 17: ['19/8/17', 'Swansea', 'Man United', 17, 0.49554188941258226],
 19: ['20/8/17', 'Tottenham', 'Chelsea', 19, 0.49917849036847],
 20: ['21/8/17', 'Man City', 'Everton', 20, 0.5075285784984221],
 21: ['26/8/17', 'Bournemouth', 'Man City', 21, 0.49189291151875447],
 22: ['26/8/17', 'Crystal Palace', 'Swansea', 22, 0.5018157678697338],
 24: ['26/8/17', 'Man United', 'Leicester', 24, 0.5021704714701138],
 25: ['26/8/17', 'Newcastle', 'West Ham', 25, 0.5382364958283407],
 27: ['27/8/17', 'Chelsea', 'Everton', 27, 0.5049503589411183],
 28: ['27/8/17', 'Liverpool', 'Arsenal', 28, 0.49678850528695045],
 29: ['27/8/17', 'Tottenham', 'Burnley', 29, 0.502307534141927],
 30: ['27/8/17', 'West Brom', 'Stoke', 30, 0.5049761983300244],
 31: ['9/9/17', 'Arsenal', 'Bournemouth', 31, 0.5036526961267248],
 33: ['9/9/17', 'Everton', 'Tottenham', 33, 0.49512590623615765],
 34: ['9/9/17', 'Leicester', 'Chelsea', 34, 0.4966879189954468],
 35: ['9/9/17', 'Man City', 'Liverpool', 35, 0.5066722277952315],
 36: ['9/9/17', 'Southampton', 'Watford', 36, 0.5067456244928891],
 37: ['9/9/17', 'Stoke', 'Man United', 37, 0.4984024126953881],
 38: ['10/9/17', 'Burnley', 'Crystal Palace', 38, 0.49941283438824285],
 39: ['10/9/17', 'Swansea', 'Newcastle', 39, 0.4572796095450231],
 42: ['16/9/17', 'Crystal Palace', 'Southampton', 42, 0.500352731867215],
 44: ['16/9/17', 'Liverpool', 'Burnley', 44, 0.49853820589936004],
 45: ['16/9/17', 'Newcastle', 'Stoke', 45, 0.5393630375360017],
 46: ['16/9/17', 'Tottenham', 'Swansea', 46, 0.5040329660360731],
 47: ['16/9/17', 'Watford', 'Man City', 47, 0.48678788664855355],
 48: ['16/9/17', 'West Brom', 'West Ham', 48, 0.5038496566223635],
 49: ['17/9/17', 'Chelsea', 'Arsenal', 49, 0.5011309283429627],
 50: ['17/9/17', 'Man United', 'Everton', 50, 0.5043055790428483],
 52: ['23/9/17', 'Everton', 'Bournemouth', 52, 0.49983326552856927],
 53: ['23/9/17', 'Leicester', 'Liverpool', 53, 0.501030342051459],
 54: ['23/9/17', 'Man City', 'Crystal Palace', 54, 0.5051200977190037],
 55: ['23/9/17', 'Southampton', 'Man United', 55, 0.4970049254151011],
 56: ['23/9/17', 'Stoke', 'Chelsea', 56, 0.4977576327971181],
 57: ['23/9/17', 'Swansea', 'Watford', 57, 0.5052825884903702],
 58: ['23/9/17', 'West Ham', 'Tottenham', 58, 0.4994572693182244],
 60: ['25/9/17', 'Arsenal', 'West Brom', 60, 0.49514158125755603],
 61: ['30/9/17', 'Bournemouth', 'Leicester', 61, 0.4975347972625269],
 62: ['30/9/17', 'Chelsea', 'Man City', 62, 0.49717336562461145],
 64: ['30/9/17', 'Man United', 'Crystal Palace', 64, 0.5018970982634299],
 65: ['30/9/17', 'Stoke', 'Southampton', 65, 0.5011490724622023],
 66: ['30/9/17', 'West Brom', 'Watford', 66, 0.5133677249212851],
 67: ['30/9/17', 'West Ham', 'Swansea', 67, 0.5037386501723821],
 69: ['1/10/17', 'Everton', 'Burnley', 69, 0.4976818551961693],
 70: ['1/10/17', 'Newcastle', 'Liverpool', 70, 0.5417115082072167],
 71: ['14/10/17', 'Burnley', 'West Ham', 71, 0.4974899520855946],
 72: ['14/10/17', 'Crystal Palace', 'Chelsea', 72, 0.49696129220213076],
 73: ['14/10/17', 'Liverpool', 'Man United', 73, 0.4960539420241731],
 74: ['14/10/17', 'Man City', 'Stoke', 74, 0.5043237571240164],
 76: ['14/10/17', 'Tottenham', 'Bournemouth', 76, 0.5044589444743269],
 77: ['14/10/17', 'Watford', 'Arsenal', 77, 0.49074544936690484],
 79: ['15/10/17', 'Southampton', 'Newcastle', 79, 0.458742645547542],
 80: ['16/10/17', 'Leicester', 'West Brom', 80, 0.4934572582321349],
 82: ['21/10/17', 'Chelsea', 'Watford', 82, 0.5101370641579732],
 84: ['21/10/17', 'Man City', 'Burnley', 84, 0.5054588485126762],
 85: ['21/10/17', 'Newcastle', 'Crystal Palace', 85, 0.540159378130989],
 86: ['21/10/17', 'Southampton', 'West Brom', 86, 0.49312948475351925],
 87: ['21/10/17', 'Stoke', 'Bournemouth', 87, 0.503038086902975],
 88: ['21/10/17', 'Swansea', 'Leicester', 88, 0.49796077570078084],
 89: ['22/10/17', 'Everton', 'Arsenal', 89, 0.4959321545837597],
 90: ['22/10/17', 'Tottenham', 'Liverpool', 90, 0.5035209134244822],
 91: ['28/10/17', 'Arsenal', 'Swansea', 91, 0.5032267176884709],
 92: ['28/10/17', 'Bournemouth', 'Chelsea', 92, 0.4944711310760584],
 93: ['28/10/17', 'Crystal Palace', 'West Ham', 93, 0.497828702879267],
 95: ['28/10/17', 'Man United', 'Tottenham', 95, 0.4996799000970907],
 96: ['28/10/17', 'Watford', 'Stoke', 96, 0.4913600585906546],
 97: ['28/10/17', 'West Brom', 'Man City', 97, 0.5004040263879234],
 99: ['29/10/17', 'Leicester', 'Everton', 99, 0.5018866927546498],
 100: ['30/10/17', 'Burnley', 'Newcastle', 100, 0.4590050414391692],
 102: ['4/11/17', 'Newcastle', 'Bournemouth', 102, 0.5426495392570614],
 103: ['4/11/17', 'Southampton', 'Burnley', 103, 0.4994891892902881],
 104: ['4/11/17', 'Stoke', 'Leicester', 104, 0.5008212989835866],
 106: ['4/11/17', 'West Ham', 'Liverpool', 106, 0.5032265975607914],
 107: ['5/11/17', 'Chelsea', 'Man United', 107, 0.5003963650801853],
 108: ['5/11/17', 'Everton', 'Watford', 108, 0.5049382903987703],
 109: ['5/11/17', 'Man City', 'Arsenal', 109, 0.5037091479002666],
 110: ['5/11/17', 'Tottenham', 'Crystal Palace', 110, 0.5019687833482546],
 111: ['18/11/17', 'Arsenal', 'Tottenham', 111, 0.49894533683431325],
 113: ['18/11/17', 'Burnley', 'Swansea', 113, 0.5014770170760614],
 114: ['18/11/17', 'Crystal Palace', 'Everton', 114, 0.5021600659613338],
 115: ['18/11/17', 'Leicester', 'Man City', 115, 0.4941096994381429],
 116: ['18/11/17', 'Liverpool', 'Southampton', 116, 0.49880060179098723],
 117: ['18/11/17', 'Man United', 'Newcastle', 117, 0.46148930531435617],
 118: ['18/11/17', 'West Brom', 'Chelsea', 118, 0.5029822459452272],
 119: ['19/11/17', 'Watford', 'West Ham', 119, 0.49023351688299366],
 121: ['24/11/17', 'West Ham', 'Leicester', 121, 0.5019478406912476],
 122: ['25/11/17', 'Crystal Palace', 'Stoke', 122, 0.49895524458692797],
 123: ['25/11/17', 'Liverpool', 'Chelsea', 123, 0.4954091621259031],
 125: ['25/11/17', 'Newcastle', 'Watford', 125, 0.5477545641272623],
 126: ['25/11/17', 'Swansea', 'Bournemouth', 126, 0.5001775636201693],
 127: ['25/11/17', 'Tottenham', 'West Brom', 127, 0.4959478296051581],
 128: ['26/11/17', 'Burnley', 'Arsenal', 128, 0.4980018845695057],
 130: ['26/11/17', 'Southampton', 'Everton', 130, 0.5015589192760341],
 132: ['28/11/17', 'Leicester', 'Tottenham', 132, 0.4972610138088921],
 133: ['28/11/17', 'Watford', 'Man United', 133, 0.4900108861041274],
 134: ['28/11/17', 'West Brom', 'Newcastle', 134, 0.4653647459759381],
 136: ['29/11/17', 'Bournemouth', 'Burnley', 136, 0.49760017484951535],
 137: ['29/11/17', 'Chelsea', 'Swansea', 137, 0.5046060608495183],
 138: ['29/11/17', 'Everton', 'West Ham', 138, 0.4954202220998486],
 139: ['29/11/17', 'Man City', 'Southampton', 139, 0.5057212444043033],
 140: ['29/11/17', 'Stoke', 'Liverpool', 140, 0.5021000558531303],
 141: ['2/12/17', 'Arsenal', 'Man United', 141, 0.4990170219191379],
 143: ['2/12/17', 'Chelsea', 'Newcastle', 143, 0.46213408521262617],
 145: ['2/12/17', 'Leicester', 'Burnley', 145, 0.49981696276890375],
 146: ['2/12/17', 'Stoke', 'Swansea', 146, 0.5026121084647212],
 147: ['2/12/17', 'Watford', 'Tottenham', 147, 0.4899392010193027],
 148: ['2/12/17', 'West Brom', 'Crystal Palace', 148, 0.5057725389250117],
 149: ['3/12/17', 'Bournemouth', 'Southampton', 149, 0.49786257074114254],
 150: ['3/12/17', 'Man City', 'West Ham', 150, 0.5031972154163554],
 151: ['9/12/17', 'Burnley', 'Watford', 151, 0.5070080203845162],
 152: ['9/12/17', 'Crystal Palace', 'Bournemouth', 152, 0.5022417463079877],
 154: ['9/12/17', 'Newcastle', 'Leicester', 154, 0.540432751337673],
 155: ['9/12/17', 'Swansea', 'West Brom', 155, 0.4916664487510004],
 156: ['9/12/17', 'Tottenham', 'Stoke', 156, 0.5011724427532672],
 157: ['9/12/17', 'West Ham', 'Chelsea', 157, 0.49888417450477907],
 158: ['10/12/17', 'Liverpool', 'Everton', 158, 0.5006079358851061],
 159: ['10/12/17', 'Man United', 'Man City', 159, 0.49652858572634145],
 160: ['10/12/17', 'Southampton', 'Arsenal', 160, 0.49773948867787854],
 161: ['12/12/17', 'Burnley', 'Stoke', 161, 0.49861649379325557],
 162: ['12/12/17', 'Crystal Palace', 'Watford', 162, 0.5073467711781887],
 164: ['13/12/17', 'Liverpool', 'West Brom', 164, 0.49217850136259117],
 165: ['13/12/17', 'Man United', 'Bournemouth', 165, 0.5043872593895022],
 166: ['13/12/17', 'Newcastle', 'Everton', 166, 0.5425678589104075],
 167: ['13/12/17', 'Southampton', 'Leicester', 167, 0.4994238117032997],
 168: ['13/12/17', 'Swansea', 'Man City', 168, 0.49231888995700845],
 170: ['13/12/17', 'West Ham', 'Arsenal', 170, 0.5002635176658264],
 171: ['16/12/17', 'Arsenal', 'Newcastle', 171, 0.46075474205157874],
 173: ['16/12/17', 'Chelsea', 'Southampton', 173, 0.5031430248469995],
 174: ['16/12/17', 'Leicester', 'Crystal Palace', 174, 0.4994782119752313],
 175: ['16/12/17', 'Man City', 'Tottenham', 175, 0.5029028995526645],
 176: ['16/12/17', 'Stoke', 'West Ham', 176, 0.49862504347425435],
 178: ['17/12/17', 'Bournemouth', 'Liverpool', 178, 0.4988135541320706],
 179: ['17/12/17', 'West Brom', 'Man United', 179, 0.5036270258434972],
 180: ['18/12/17', 'Everton', 'Swansea', 180, 0.49940728709031534],
 181: ['22/12/17', 'Arsenal', 'Liverpool', 181, 0.5027146650768801],
 183: ['23/12/17', 'Burnley', 'Tottenham', 183, 0.49719563622190366],
 184: ['23/12/17', 'Everton', 'Chelsea', 184, 0.49455281142271235],
 185: ['23/12/17', 'Leicester', 'Man United', 185, 0.49733269889371673],
 186: ['23/12/17', 'Man City', 'Bournemouth', 186, 0.5076102588450762],
 188: ['23/12/17', 'Stoke', 'West Brom', 188, 0.4945269720338062],
 189: ['23/12/17', 'Swansea', 'Crystal Palace', 189, 0.4976874024940968],
 190: ['23/12/17', 'West Ham', 'Newcastle', 190, 0.4612666745354899],
 191: ['26/12/17', 'Bournemouth', 'West Ham', 191, 0.4953385417531946],
 194: ['26/12/17', 'Liverpool', 'Swansea', 194, 0.5002636377935061],
 195: ['26/12/17', 'Man United', 'Burnley', 195, 0.5022358490571023],
 196: ['26/12/17', 'Tottenham', 'Southampton', 196, 0.5025699300335542],
 197: ['26/12/17', 'Watford', 'Leicester', 197, 0.49242977239232594],
 198: ['26/12/17', 'West Brom', 'Everton', 198, 0.5081810197044302],
 199: ['27/12/17', 'Newcastle', 'Man City', 199, 0.5347908655939005],
 200: ['28/12/17', 'Crystal Palace', 'Arsenal', 200, 0.4983406353631782],
 201: ['30/12/17', 'Bournemouth', 'Everton', 201, 0.49966990483526136],
 202: ['30/12/17', 'Chelsea', 'Stoke', 202, 0.5017455375667126],
 204: ['30/12/17', 'Liverpool', 'Leicester', 204, 0.4984728283123716],
 205: ['30/12/17', 'Man United', 'Southampton', 205, 0.5024982449487295],
 207: ['30/12/17', 'Watford', 'Swansea', 207, 0.4942205818734604],
 208: ['31/12/17', 'Crystal Palace', 'Man City', 208, 0.4943830726448269],
 209: ['31/12/17', 'West Brom', 'Arsenal', 209, 0.5043615891062746],
 211: ['1/1/18', 'Burnley', 'Liverpool', 211, 0.5009649644644706],
 212: ['1/1/18', 'Everton', 'Man United', 212, 0.4951975913209823],
 214: ['1/1/18', 'Stoke', 'Newcastle', 214, 0.46014013282782895],
 215: ['2/1/18', 'Man City', 'Watford', 215, 0.5127152837152771],
 216: ['2/1/18', 'Southampton', 'Crystal Palace', 216, 0.49915043849661567],
 217: ['2/1/18', 'Swansea', 'Tottenham', 217, 0.4954702043277576],
 218: ['2/1/18', 'West Ham', 'West Brom', 218, 0.49565351374146716],
 219: ['3/1/18', 'Arsenal', 'Chelsea', 219, 0.49837224202086794],
 220: ['4/1/18', 'Tottenham', 'West Ham', 220, 0.5000459010456062],
 221: ['13/1/18', 'Chelsea', 'Leicester', 221, 0.5028152513683839],
 222: ['13/1/18', 'Crystal Palace', 'Burnley', 222, 0.5000903359755877],
 224: ['13/1/18', 'Newcastle', 'Swansea', 224, 0.5422235608188075],
 225: ['13/1/18', 'Tottenham', 'Everton', 225, 0.5043772641276729],
 226: ['13/1/18', 'Watford', 'Southampton', 226, 0.4927575458709416],
 228: ['14/1/18', 'Bournemouth', 'Arsenal', 228, 0.49585047423710576],
 229: ['14/1/18', 'Liverpool', 'Man City', 229, 0.4928309425685992],
 230: ['15/1/18', 'Man United', 'Stoke', 230, 0.5011007576684425],
 231: ['20/1/18', 'Arsenal', 'Crystal Palace', 231, 0.5011625350006524],
 233: ['20/1/18', 'Burnley', 'Man United', 233, 0.4972673213067283],
 234: ['20/1/18', 'Everton', 'West Brom', 234, 0.49132215065940044],
 235: ['20/1/18', 'Leicester', 'Watford', 235, 0.5070733979715046],
 236: ['20/1/18', 'Man City', 'Newcastle', 236, 0.46471230476993003],
 238: ['20/1/18', 'West Ham', 'Bournemouth', 238, 0.504164628610636],
 239: ['21/1/18', 'Southampton', 'Tottenham', 239, 0.49693324033027647],
 240: ['22/1/18', 'Swansea', 'Liverpool', 240, 0.49923953257032455],
 242: ['30/1/18', 'Swansea', 'Arsenal', 242, 0.4962764526753597],
 243: ['30/1/18', 'West Ham', 'Crystal Palace', 243, 0.5016744674845636],
 244: ['31/1/18', 'Chelsea', 'Bournemouth', 244, 0.5050320392877723],
 245: ['31/1/18', 'Everton', 'Leicester', 245, 0.4976164776091809],
 246: ['31/1/18', 'Man City', 'West Brom', 246, 0.49909914397590727],
 247: ['31/1/18', 'Newcastle', 'Burnley', 247, 0.5404981289246614],
 249: ['31/1/18', 'Stoke', 'Watford', 249, 0.508143111773176],
 250: ['31/1/18', 'Tottenham', 'Man United', 250, 0.49982327026673995],
 251: ['3/2/18', 'Arsenal', 'Everton', 251, 0.5035710157800709],
 252: ['3/2/18', 'Bournemouth', 'Stoke', 252, 0.4964650834608556],
 254: ['3/2/18', 'Burnley', 'Man City', 254, 0.4940443218511545],
 255: ['3/2/18', 'Leicester', 'Swansea', 255, 0.5015423946630498],
 257: ['3/2/18', 'West Brom', 'Southampton', 257, 0.5063736856103114],
 258: ['4/2/18', 'Crystal Palace', 'Newcastle', 258, 0.4593437922328416],
 259: ['4/2/18', 'Liverpool', 'Tottenham', 259, 0.4959822569393484],
 260: ['5/2/18', 'Watford', 'Chelsea', 260, 0.4893661062058574],
 261: ['10/2/18', 'Everton', 'Crystal Palace', 261, 0.49734310440249685],
 262: ['10/2/18', 'Man City', 'Leicester', 262, 0.5053934709256878],
 264: ['10/2/18', 'Swansea', 'Burnley', 264, 0.4980261532877693],
 265: ['10/2/18', 'Tottenham', 'Arsenal', 265, 0.5005578335295174],
 266: ['10/2/18', 'West Ham', 'Watford', 266, 0.509269653480837],
 268: ['11/2/18', 'Newcastle', 'Man United', 268, 0.5380138650494745],
 269: ['11/2/18', 'Southampton', 'Liverpool', 269, 0.5007025685728433],
 270: ['12/2/18', 'Chelsea', 'West Brom', 270, 0.4965209244186034],
 271: ['24/2/18', 'Bournemouth', 'Newcastle', 271, 0.45685363110676924],
 273: ['24/2/18', 'Burnley', 'Southampton', 273, 0.5000139810735426],
 274: ['24/2/18', 'Leicester', 'Stoke', 274, 0.498681871380244],
 275: ['24/2/18', 'Liverpool', 'West Ham', 275, 0.49627657280303933],
 276: ['24/2/18', 'Watford', 'Everton', 276, 0.4945648799650604],
 278: ['25/2/18', 'Crystal Palace', 'Tottenham', 278, 0.4975343870155761],
 279: ['25/2/18', 'Man United', 'Chelsea', 279, 0.4991068052836453],
 280: ['1/3/18', 'Arsenal', 'Man City', 280, 0.495794022463564],
 281: ['3/3/18', 'Burnley', 'Everton', 281, 0.5018213151676613],
 282: ['3/3/18', 'Leicester', 'Bournemouth', 282, 0.5019683731013037],
 283: ['3/3/18', 'Liverpool', 'Newcastle', 283, 0.45779166215661393],
 284: ['3/3/18', 'Southampton', 'Stoke', 284, 0.4983540979016284],
 285: ['3/3/18', 'Swansea', 'West Ham', 285, 0.4957645201914485],
 287: ['3/3/18', 'Watford', 'West Brom', 287, 0.4861354454425455],
 289: ['4/3/18', 'Man City', 'Chelsea', 289, 0.5023298047392192],
 290: ['5/3/18', 'Crystal Palace', 'Man United', 290, 0.49760607210040075],
 291: ['10/3/18', 'Chelsea', 'Crystal Palace', 291, 0.5025418781616998],
 294: ['10/3/18', 'Man United', 'Liverpool', 294, 0.5034492283396576],
 295: ['10/3/18', 'Newcastle', 'Southampton', 295, 0.5407605248162887],
 296: ['10/3/18', 'West Brom', 'Leicester', 296, 0.5060459121316957],
 297: ['10/3/18', 'West Ham', 'Burnley', 297, 0.5020132182782361],
 298: ['11/3/18', 'Arsenal', 'Watford', 298, 0.5087577209969258],
 299: ['11/3/18', 'Bournemouth', 'Tottenham', 299, 0.4950442258895037],
 300: ['12/3/18', 'Stoke', 'Man City', 300, 0.49517941323981424],
 301: ['17/3/18', 'Bournemouth', 'West Brom', 301, 0.4912404703127465],
 303: ['17/3/18', 'Liverpool', 'Watford', 303, 0.5057946411019609],
 304: ['17/3/18', 'Stoke', 'Everton', 304, 0.5029564065563211],
 306: ['31/3/18', 'Crystal Palace', 'Liverpool', 306, 0.501303715258143],
 307: ['31/3/18', 'Everton', 'Man City', 307, 0.4919745918654085],
 308: ['31/3/18', 'Man United', 'Swansea', 308, 0.5039612809512484],
 310: ['31/3/18', 'Watford', 'Bournemouth', 310, 0.4946465603117144],
 311: ['31/3/18', 'West Brom', 'Burnley', 311, 0.5061112897186841],
 312: ['31/3/18', 'West Ham', 'Southampton', 312, 0.5022756141698632],
 313: ['1/4/18', 'Arsenal', 'Stoke', 313, 0.5003661944056651],
 314: ['1/4/18', 'Chelsea', 'Tottenham', 314, 0.5003246799953607],
 315: ['7/4/18', 'Bournemouth', 'Crystal Palace', 315, 0.4972614240558429],
 317: ['7/4/18', 'Everton', 'Liverpool', 317, 0.4988952344787246],
 318: ['7/4/18', 'Leicester', 'Newcastle', 318, 0.45907041902615764],
 319: ['7/4/18', 'Man City', 'Man United', 319, 0.5029745846374891],
 320: ['7/4/18', 'Stoke', 'Tottenham', 320, 0.4983307276105634],
 321: ['7/4/18', 'Watford', 'Burnley', 321, 0.4924951499793144],
 322: ['7/4/18', 'West Brom', 'Swansea', 322, 0.5078367216128302],
 323: ['8/4/18', 'Arsenal', 'Southampton', 323, 0.5017636816859521],
 324: ['8/4/18', 'Chelsea', 'West Ham', 324, 0.5006189958590516],
 325: ['14/4/18', 'Burnley', 'Leicester', 325, 0.4996862075949269],
 328: ['14/4/18', 'Liverpool', 'Bournemouth', 328, 0.50068961623176],
 329: ['14/4/18', 'Southampton', 'Chelsea', 329, 0.49636014551683116],
 330: ['14/4/18', 'Swansea', 'Everton', 330, 0.5000958832735153],
 331: ['14/4/18', 'Tottenham', 'Man City', 331, 0.49660027081116614],
 332: ['15/4/18', 'Man United', 'West Brom', 332, 0.49587614452033346],
 333: ['15/4/18', 'Newcastle', 'Arsenal', 333, 0.5387484283122519],
 334: ['16/4/18', 'West Ham', 'Stoke', 334, 0.5008781268895763],
 336: ['18/4/18', 'Bournemouth', 'Man United', 336, 0.49511591097432833],
 337: ['19/4/18', 'Burnley', 'Chelsea', 337, 0.49662254140845835],
 338: ['19/4/18', 'Leicester', 'Southampton', 338, 0.500079358660531],
 339: ['21/4/18', 'Watford', 'Crystal Palace', 339, 0.49215639918564197],
 340: ['21/4/18', 'West Brom', 'Liverpool', 340, 0.5073246690012394],
 341: ['22/4/18', 'Arsenal', 'West Ham', 341, 0.49923965269800413],
 342: ['22/4/18', 'Man City', 'Swansea', 342, 0.5071842804068222],
 343: ['22/4/18', 'Stoke', 'Burnley', 343, 0.5008866765705751],
 344: ['23/4/18', 'Everton', 'Newcastle', 344, 0.4569353114534232],
 346: ['28/4/18', 'Crystal Palace', 'Leicester', 346, 0.5000249583885993],
 348: ['28/4/18', 'Liverpool', 'Stoke', 348, 0.4974031145107003],
 349: ['28/4/18', 'Newcastle', 'West Brom', 349, 0.5341384243878926],
 350: ['28/4/18', 'Southampton', 'Bournemouth', 350, 0.501640599622688],
 351: ['28/4/18', 'Swansea', 'Chelsea', 351, 0.4948971095143123],
 352: ['29/4/18', 'Man United', 'Arsenal', 352, 0.5004861484446927],
 353: ['29/4/18', 'West Ham', 'Man City', 353, 0.4963059549474752],
 354: ['30/4/18', 'Tottenham', 'Watford', 354, 0.5095639693445279],
 356: ['5/5/18', 'Bournemouth', 'Swansea', 356, 0.4993256067436614],
 357: ['5/5/18', 'Everton', 'Southampton', 357, 0.4979442510877965],
 358: ['5/5/18', 'Leicester', 'West Ham', 358, 0.49755532967258304],
 359: ['5/5/18', 'Stoke', 'Crystal Palace', 359, 0.5005479257769027],
 360: ['5/5/18', 'Watford', 'Newcastle', 360, 0.45174860623656826],
 361: ['5/5/18', 'West Brom', 'Tottenham', 361, 0.5035553407586726],
 362: ['6/5/18', 'Arsenal', 'Burnley', 362, 0.5015012857943248],
 363: ['6/5/18', 'Chelsea', 'Liverpool', 363, 0.5040940082379276],
 365: ['8/5/18', 'Swansea', 'Southampton', 365, 0.49828854917939647],
 367: ['9/5/18', 'Leicester', 'Arsenal', 367, 0.49806726215649416],
 369: ['9/5/18', 'Tottenham', 'Newcastle', 369, 0.46156099039918086],
 370: ['10/5/18', 'West Ham', 'Man United', 370, 0.49952895440304906],
 371: ['13/5/18', 'Man United', 'Watford', 371, 0.5094922842597033],
 372: ['13/5/18', 'Newcastle', 'Chelsea', 372, 0.5373690851512044],
 373: ['13/5/18', 'Southampton', 'Man City', 373, 0.49378192595952725],
 374: ['13/5/18', 'Swansea', 'Stoke', 374, 0.49689106189910953],
 377: ['13/5/18', 'Burnley', 'Bournemouth', 377, 0.5019029955143153],
 378: ['13/5/18', 'Crystal Palace', 'West Brom', 378, 0.4937306314388189],
 379: ['13/5/18', 'Tottenham', 'Leicester', 379, 0.5022421565549385]}

In [ ]:
# dfA = pd.DataFrame.from_dict(dfA,orient='index')
dfAF = pd.DataFrame.from_dict(dfA)
dfAF2 = dfAF.T

In [ ]:
dfAF3 = dfAF2.reset_index(drop=True)

In [ ]:
ptdf8.set_index("Game_ID").head()
ptdf9 = ptdf8.reset_index(drop=True)

In [ ]:
ptdf9["AT win Predict"] = dfAF3.loc[:, 4]

In [ ]:
def win(row):
    if (row["HT win Predict"] > row["AT win Predict"]):
        return row["HomeTeam"]
    elif (row["AT win Predict"] > row["HT win Predict"]):
        return row["AwayTeam"]
    elif (row["AT win Predict"] == row["HT win Predict"]):
        return "DRAW"
    

def lose(row):
    if (row["HT win Predict"] < row["AT win Predict"]):
        return row["HomeTeam"]
    elif (row["AT win Predict"] < row["HT win Predict"]):
        return row["AwayTeam"]
    elif (row["AT win Predict"] == row["HT win Predict"]):
        return "DRAW"

ptdf9["Winner"] = ptdf9.apply(win, axis=1) 
ptdf9["Loser"] = ptdf9.apply(lose, axis=1)

In [ ]:
ptdf7 = ptdf6[ptdf6.HomeTeam.str.contains("Newcastle") == False]
ptdf8 = ptdf7[ptdf7.AwayTeam.str.contains("Newcastle") == False]
ptdf9["Winner"].value_counts()
ptdf9["Loser"].value_counts()